In [1]:
import numpy as np

In [2]:
input_data=np.load("CK+dataset.npy")
label=np.load("CK+label.npy")

In [3]:
label=label-1

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

In [5]:
input_data=torch.FloatTensor(input_data)
label=torch.LongTensor(label)

In [6]:
input_data.shape

torch.Size([327, 640, 480])

In [7]:
remake=input_data.unsqueeze(0)
remake=remake.transpose(0,1)

In [8]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
dataset = TensorDataset(remake,label)

In [9]:
trainset,valset=torch.utils.data.random_split(dataset,[277,50])
trainset,testset=torch.utils.data.random_split(trainset,[227,50])

partition={'train':trainset,'val':valset,'test':testset}

In [26]:
class AlexNet(nn.Module):
    
    def __init__(self):
        super(AlexNet,self).__init__()
        self.feature=nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),# 64 56 56
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3,stride=2),#MAXPooling 할 때 마지막 1은 버림. 64 27 27
        nn.Conv2d(64, 192, kernel_size=5, padding=2), # 192, 27, 27
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3,stride=2), # 192 13 13
        nn.Conv2d(192, 384, kernel_size=3, padding=1), # 384 13 13
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 256, kernel_size=3, padding=1), # 256 13 13
        nn.ReLU(inplace=True),
        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2) # 256 6 6 
        )
        self.avgpool=nn.AdaptiveAvgPool2d((6,6)) # 256 6 6 (6,6)은 output size이다.
        # 평균을 구하는 것으로 반올림 정수가 나옴.
        self.classifier = nn.Sequential(
        nn.Dropout(),
        nn.Linear(256*6*6,4096),  # 9216=>4096, 256 36
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(4096,4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096,7), # => 2 class
        
        )
    def forward(self, x):
        x=self.feature(x)
        x=self.avgpool(x)
        x=torch.flatten(x,1) # ex => 2,2,2 => 2,4
        x=self.classifier(x)
        return x

In [27]:
def dimension_check():
    net=AlexNet()
    x= torch.randn(2,1,640,480)
    y=net(x)
    print(y.size())

In [28]:
dimension_check()

torch.Size([2, 7])


In [29]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=32,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [30]:
def test(net, partition):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=32, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [31]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=32, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [32]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [36]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net=AlexNet()
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.015, weight_decay=0.001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(50):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
test_acc = test(net,partition)
test_acc=round(test_acc,2)

 25%|██▌       | 2/8 [00:00<00:00, 12.19it/s]

time : 0.76, train_loss : 649742.90, train_acc : 12.78,val_acc : 24.00,val_loss : 194.56


 25%|██▌       | 2/8 [00:00<00:00, 12.26it/s]

time : 0.73, train_loss : 36.71, train_acc : 17.18,val_acc : 16.00,val_loss : 2.01


 25%|██▌       | 2/8 [00:00<00:00, 12.30it/s]

...

 25%|██▌       | 2/8 [00:00<00:00, 12.22it/s]

time : 0.75, train_loss : 1.87, train_acc : 24.23,val_acc : 24.00,val_loss : 1.78


  0%|          | 0/2 [00:00<?, ?it/s]

time : 0.72, train_loss : 1.81, train_acc : 22.91,val_acc : 28.00,val_loss : 1.79


100%|██████████| 2/2 [00:00<00:00, 14.59it/s]


In [37]:
np.save("/home/lab/양창희/Emotion Project/test_list_file/Alex_train_loss.npy",train_loss)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Alex_val_loss.npy",val_loss)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Alex_train_acc.npy",train_acc)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Alex_val_acc.npy",val_acc)